In [1]:
import os
import pickle
from pathlib import Path
from typing import Generator

import numpy as np
import pandas as pd
import scikit_posthocs as sp
from IPython.display import HTML, display
from scipy import stats
from yaml import safe_load
from jmetal.lab.statistical_test.functions import (
    friedman_aligned_rank_test,
    friedman_aligned_ph_test,
)


pd.set_option("display.max_columns", 100)


def friedmann_nemenyi_test(
    unravelled_detailed_results: dict[str : dict[tuple[str, str], list[float]]],
) -> None:
    for metric in unravelled_detailed_results.keys():
        metric_df = pd.DataFrame(unravelled_detailed_results[metric])
        metric_vals = [metric_df[col].values for col in metric_df.columns]
        out = stats.friedmanchisquare(*metric_vals)
        print(
            f"Friedman Test {metric}: statistic={out.statistic:.3f}, pvalue={out.pvalue:.3f}"
        )
        metric_vals = np.array(metric_vals).T
        out = sp.posthoc_nemenyi_friedman(metric_vals)
        print("Nemenyi post-hoc test")
        feature_names = metric_df.columns
        out.index = feature_names
        out.columns = feature_names
        out = out.sort_index(axis=0).sort_index(axis=1)
        display(out)


def aligned_friedmann_holm_test(
    unravelled_detailed_results: dict[str : dict[tuple[str, str], list[float]]],
) -> None:
    for metric in unravelled_detailed_results.keys():
        metric_df = pd.DataFrame(unravelled_detailed_results[metric])
        # metric_vals = np.asarray([metric_df[col].values for col in metric_df.columns])
        metric_vals = metric_df.values

        out = friedman_aligned_rank_test(metric_vals)
        chi2_stat = out.loc["Aligned Rank stat"].iloc[0]
        p_value = out.loc["p-value"].iloc[0]
        print(f"Aligned-rank Friedman χ² {metric} = {chi2_stat:.3f}, p = {p_value:.3f}")

        z_vals, out, _ = friedman_aligned_ph_test(
            metric_vals, apv_procedure="Holm"  # Holm step-down correction
        )
        print("Holm post-hoc test")
        feature_names = metric_df.columns
        out.index = feature_names
        out.columns = feature_names
        out = out.sort_index(axis=0).sort_index(axis=1)
        display(out)


def present_results(
    paths: Generator,
    val_method: str = "lopo",
    remove_xgboost: bool = False,
    remove_chronos_small_from_test: bool = False,
    which_test: str = "friedmann-nemenyi",
    test_args: dict = {},
) -> None:
    results = []
    for reports_path in paths:
        report = pd.read_csv(reports_path, index_col=0)
        conf = safe_load(open(reports_path.parent / ".hydra/config.yaml"))
        if conf["validation_method"]["_target_"].split(".")[-1].lower() != val_method:
            continue
        model_name: str = conf["model"]["model"]["_target_"].split(".")[-1]
        features_name = (
            conf["feature_extractor"]["_target_"].split(".")[-1]
            if "model_name" not in conf["feature_extractor"]
            else conf["feature_extractor"]["model_name"]
        )
        validation_method = conf["validation_method"]["_target_"].split(".")[-1]
        if "aggregator" not in conf:
            aggregator = "MeanTimeAggregator"
        else:
            aggregator = (
                conf["aggregator"]["_target_"].split(".")[-1]
                if "_target_" in conf["aggregator"]
                else None
            )
        report_results = {}
        for col in report.columns:
            report_results[f"{col} avg"] = report[col].mean()
            report_results[f"{col} sem"] = report[col].sem() * 1.98  # 95% CI

        dataset = conf["dataset"]
        side = conf["side"]
        resampling = (
            conf["resampling"]["_target_"].split(".")[-1]
            if "resampling" in conf
            else "None"
        )
        resampling = resampling if resampling != "NoUnderSampler" else "None"
        # Collect results in a list of dicts
        if remove_xgboost and model_name == "XGBClassifier":
            continue
        results.append(
            {
                "Dataset": dataset,
                "Side": side,
                "Model": model_name,
                "Resampling": resampling,
                "Features": features_name,
                "Aggregator": aggregator,
                "Validation": validation_method,
                "Detailed Report": report,
                **report_results,
            }
        )

    # After the loop, display as a table
    df_results = pd.DataFrame(results)
    for (dataset, side, resampling), group in df_results.groupby(
        ["Dataset", "Side", "Resampling"]
    ):
        try:
            display(
                HTML(
                    f"""
                <div style='background-color:#ffe6e6; padding:18px; margin:10px 0; border-radius:8px;'>
                    <h2 style='color:#b30000; margin:0; font-size:2em;'>
                    Results for Dataset: <i>{dataset}</i>, Side: <i>{side}</i>, resampling: <i>{resampling}</i>
                    </h2>
                </div>
                """
                )
            )
            grouped_data = (
                group.sort_values(by=["Model", "Features", "Aggregator"])
                .drop(columns=["Detailed Report"])
                .drop_duplicates()
            )
            # display(grouped_data)

            def test_fn(x):
                if len(x) > 1:
                    return pd.Series(
                        [
                            "%.2f ± %.2f" % (np.round(x.values[0][i], 2), np.round(x.values[1][i], 2))
                            for i in range(x.shape[1])
                        ],
                        index=x.columns,
                    )
                else:
                    return pd.Series(
                        [f"{x.values[0][i]}" for i in range(x.shape[1])],
                        index=x.columns,
                    )

            grouped_data.columns = pd.MultiIndex.from_tuples(
                [
                    (
                        tuple(col.split(" "))
                        if "avg" in col or "sem" in col
                        else tuple([col])
                    )
                    for col in grouped_data.columns
                ]
            )
            grouped_data = grouped_data.T.groupby(level=0).apply(test_fn).T
            display(grouped_data)
            grouped_data = grouped_data.drop(columns=['accuracy_score', 'Dataset', "Side", "Resampling", "Validation"])
            grouped_data = grouped_data.rename(
                columns={
                    "balanced_accuracy_score": "Balanced Accuracy",
                    "f1_score": "F1",
                    "matthews_corrcoef": "MCC",
                    "roc_auc_score": "ROC AUC",
                    "precision_score": "Precision",
                    "recall_score": "Recall",
                }
            )


            # save to latex
            latex_path = Path("../tables_latex") / f"results_{dataset}_{side}_{resampling}_{validation_method}.tex"
            with open(latex_path, "w") as f:
                f.write(grouped_data.to_latex())  

            if "aggregator" in test_args.keys() and group["Aggregator"].nunique() > 2:
                group = group[
                    (group["Aggregator"] == test_args["aggregator"])
                    | (group["Aggregator"].isnull())
                ]

            unravelled_detailed_results = {
                metric: {} for metric in group["Detailed Report"].iloc[0].columns
            }
            for i, model_results in group.iterrows():
                for metric in model_results["Detailed Report"].columns:
                    cv_results = model_results["Detailed Report"][metric]
                    unravelled_detailed_results[metric][
                        (model_results["Model"], model_results["Features"])
                    ] = cv_results

            if which_test == "friedmann-nemenyi":
                friedmann_nemenyi_test(unravelled_detailed_results)
            elif which_test == "alignedfriedmann-holm":
                aligned_friedmann_holm_test(unravelled_detailed_results)
            else:
                raise ValueError(f"Unknown test: {which_test}")
        except Exception as e:
            print(f"Error processing group {dataset}, {side}, {resampling}: {e}")
            continue

    return results

In [2]:
results_path: str = "../outputs/"
all_results = list(Path(results_path).glob("*/*/*/reports.csv"))
results_path_adula: str = "../outputs_adula/"
all_results += list(Path(results_path_adula).glob("*/*/*/reports.csv"))


results_lopo = present_results(
    all_results,
    val_method="lopo",
    remove_xgboost=True,
    remove_chronos_small_from_test=True,
    which_test="friedmann-nemenyi",
    test_args={
        "aggregator": "MeanChanAggregator"}
)

,Aggregator,Dataset,Features,Model,Resampling,Side,Validation,accuracy_score,balanced_accuracy_score,f1_score,matthews_corrcoef,precision_score,recall_score,roc_auc_score
3,None,apsync,NoneFeatureExtractor,LogisticRegression,None,engagement_10sec_35thresh,LOPO,0.65 ± 0.18,0.57 ± 0.14,0.77 ± 0.14,0.00 ± 0.02,0.66 ± 0.18,0.99 ± 0.01,0.50 ± 0.00


Error processing group apsync, engagement_10sec_35thresh, None: At least 3 sets of samples must be given for Friedman test, got 1.


,Aggregator,Dataset,Features,Model,Resampling,Side,Validation,accuracy_score,balanced_accuracy_score,f1_score,matthews_corrcoef,precision_score,recall_score,roc_auc_score
51,None,apsync,HandcraftedFeatureExtractor,DummyClassifier,None,engagement_10sec_4thresh,LOPO,0.56 ± 0.12,0.49 ± 0.01,0.67 ± 0.12,-0.01 ± 0.02,0.65 ± 0.18,0.78 ± 0.16,0.49 ± 0.01
54,MeanChanAggregator,apsync,AutonLab/MOMENT-1-large,LogisticRegression,None,engagement_10sec_4thresh,LOPO,0.58 ± 0.12,0.53 ± 0.07,0.71 ± 0.10,-0.02 ± 0.03,0.65 ± 0.18,0.88 ± 0.07,0.49 ± 0.01
50,None,apsync,HandcraftedFeatureExtractor,LogisticRegression,None,engagement_10sec_4thresh,LOPO,0.65 ± 0.17,0.56 ± 0.13,0.76 ± 0.13,-0.01 ± 0.04,0.66 ± 0.18,0.98 ± 0.01,0.50 ± 0.01
4,None,apsync,NoneFeatureExtractor,LogisticRegression,None,engagement_10sec_4thresh,LOPO,0.65 ± 0.18,0.57 ± 0.14,0.77 ± 0.14,0.00 ± 0.02,0.66 ± 0.18,0.99 ± 0.01,0.50 ± 0.00
49,MeanChanAggregator,apsync,amazon/chronos-t5-large,LogisticRegression,None,engagement_10sec_4thresh,LOPO,0.58 ± 0.13,0.51 ± 0.07,0.71 ± 0.12,-0.04 ± 0.05,0.65 ± 0.18,0.86 ± 0.09,0.48 ± 0.03
53,MeanChanAggregator,apsync,amazon/chronos-t5-small,LogisticRegression,None,engagement_10sec_4thresh,LOPO,0.58 ± 0.12,0.54 ± 0.08,0.70 ± 0.11,-0.00 ± 0.05,0.65 ± 0.18,0.82 ± 0.07,0.50 ± 0.02
52,MeanChanAggregator,apsync,ibm-granite/granite-timeseries-patchtsmixer,LogisticRegression,None,engagement_10sec_4thresh,LOPO,0.66 ± 0.18,0.57 ± 0.14,0.77 ± 0.14,-0.00 ± 0.01,0.66 ± 0.18,1.00 ± 0.00,0.50 ± 0.00
55,None,apsync,paris-noah/Mantis-8M,LogisticRegression,None,engagement_10sec_4thresh,LOPO,0.55 ± 0.13,0.50 ± 0.05,0.68 ± 0.10,0.00 ± 0.12,0.65 ± 0.19,0.83 ± 0.13,0.50 ± 0.06


Friedman Test accuracy_score: statistic=9.368, pvalue=0.227
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.983489   
                   HandcraftedFeatureExtractor                                    0.988441   
                   NoneFeatureExtractor                                           0.958863   
                   amazon/chronos-t5-large                                        0.999995   
                   amazon/chronos-t5-small                                        1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.977064   
                   paris-noah/Mantis-8M                                           0.992155   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.983489   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.619891   
                   NoneFeatureExtractor                                       0.469151   
                   amazon/chronos-t5-large                                    0.932143   
                   amazon/chronos-t5-small                                    0.994859   
                   ibm-granite/granite-timeseries-patchtsmixer                0.544321   
                   paris-noah/Mantis-8M                                       1.000000   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.988441   
LogisticRegression AutonLab/MOMENT-1-large                                        0.619891   
                   HandcraftedFeatureExtractor                                    1.000000   
                   NoneFeatureExtractor                                           0.999999   
                   amazon/chronos-t5-large                                        0.998887   
                   amazon/chronos-t5-small                                        0.968930   
                   ibm-granite/granite-timeseries-patchtsmixer                    1.000000   
                   paris-noah/Mantis-8M                                           0.693121   

                                                                                     \
                                                               NoneFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                             0.958863   
LogisticRegression AutonLab/MOMENT-1-large                                 0.469151   
                   HandcraftedFeatureExtractor                             0.999999   
                   NoneFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                 0.992155   
                   amazon/chronos-t5-small                                 0.915185   
                   ibm-granite/granite-timeseries-patchtsmixer             1.000000   
                   paris-noah/Mantis-8M                                    0.544321   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.999995   
LogisticRegression AutonLab/MOMENT-1-large                                    0.932143   
                   HandcraftedFeatureExtractor               

...

DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.182775   
                   HandcraftedFeatureExtractor                                    0.000288   
                   NoneFeatureExtractor                                           0.000288   
                   amazon/chronos-t5-large                                        0.000256   
                   amazon/chronos-t5-small                                        0.000919   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.000108   
                   paris-noah/Mantis-8M                                           0.001975   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.182775   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.571004   
                   NoneFeatureExtractor                                       0.571004   
                   amazon/chronos-t5-large                                    0.552689   
                   amazon/chronos-t5-small                                    0.746005   
                   ibm-granite/granite-timeseries-patchtsmixer                0.426715   
                   paris-noah/Mantis-8M                                       0.846889   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.000288   
LogisticRegression AutonLab/MOMENT-1-large                                        0.571004   
                   HandcraftedFeatureExtractor                                    1.000000   
                   NoneFeatureExtractor                                           1.000000   
                   amazon/chronos-t5-large                                        1.000000   
                   amazon/chronos-t5-small                                        0.999996   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.999999   
                   paris-noah/Mantis-8M                                           0.999839   

                                                                                     \
                                                               NoneFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                             0.000288   
LogisticRegression AutonLab/MOMENT-1-large                                 0.571004   
                   HandcraftedFeatureExtractor                             1.000000   
                   NoneFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                 1.000000   
                   amazon/chronos-t5-small                                 0.999996   
                   ibm-granite/granite-timeseries-patchtsmixer             0.999999   
                   paris-noah/Mantis-8M                                    0.999839   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.000256   
LogisticRegression AutonLab/MOMENT-1-large                                    0.552689   
                   HandcraftedFeatureExtractor               

Friedman Test recall_score: statistic=55.684, pvalue=0.000
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.984010   
                   HandcraftedFeatureExtractor                                    0.000065   
                   NoneFeatureExtractor                                           0.497935   
                   amazon/chronos-t5-large                                        0.217023   
                   amazon/chronos-t5-small                                        0.229335   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.001775   
                   paris-noah/Mantis-8M                                           0.078230   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                            9.840101e-01   
LogisticRegression AutonLab/MOMENT-1-large                                1.000000e+00   
                   HandcraftedFeatureExtractor                            4.754837e-07   
                   NoneFeatureExtractor                                   7.278964e-02   
                   amazon/chronos-t5-large                                1.700433e-02   
                   amazon/chronos-t5-small                                1.857439e-02   
                   ibm-granite/granite-timeseries-patchtsmixer            2.610858e-05   
                   paris-noah/Mantis-8M                                   3.693698e-03   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                6.521091e-05   
LogisticRegression AutonLab/MOMENT-1-large                                    4.754837e-07   
                   HandcraftedFeatureExtractor                                1.000000e+00   
                   NoneFeatureExtractor                                       1.105769e-01   
                   amazon/chronos-t5-large                                    3.123791e-01   
                   amazon/chronos-t5-small                                    2.974837e-01   
                   ibm-granite/granite-timeseries-patchtsmixer                9.958502e-01   
                   paris-noah/Mantis-8M                                       5.892887e-01   

                                                                                     \
                                                               NoneFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                             0.497935   
LogisticRegression AutonLab/MOMENT-1-large                                 0.072790   
                   HandcraftedFeatureExtractor                             0.110577   
                   NoneFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                 0.999764   
                   amazon/chronos-t5-small                                 0.999839   
                   ibm-granite/granite-timeseries-patchtsmixer             0.479870   
                   paris-noah/Mantis-8M                                    0.986508   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.217023   
LogisticRegression AutonLab/MOMENT-1-large                                    0.017004   
                   HandcraftedFeatureExtractor               

Friedman Test roc_auc_score: statistic=32.411, pvalue=0.000
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.949652   
                   HandcraftedFeatureExtractor                                    0.001148   
                   NoneFeatureExtractor                                           0.009833   
                   amazon/chronos-t5-large                                        0.033637   
                   amazon/chronos-t5-small                                        0.006088   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.001593   
                   paris-noah/Mantis-8M                                           0.030983   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.949652   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.062839   
                   NoneFeatureExtractor                                       0.242093   
                   amazon/chronos-t5-large                                    0.461957   
                   amazon/chronos-t5-small                                    0.182775   
                   ibm-granite/granite-timeseries-patchtsmixer                0.078230   
                   paris-noah/Mantis-8M                                       0.444228   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.001148   
LogisticRegression AutonLab/MOMENT-1-large                                        0.062839   
                   HandcraftedFeatureExtractor                                    1.000000   
                   NoneFeatureExtractor                                           0.999344   
                   amazon/chronos-t5-large                                        0.984010   
                   amazon/chronos-t5-small                                        0.999893   
                   ibm-granite/granite-timeseries-patchtsmixer                    1.000000   
                   paris-noah/Mantis-8M                                           0.986508   

                                                                                     \
                                                               NoneFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                             0.009833   
LogisticRegression AutonLab/MOMENT-1-large                                 0.242093   
                   HandcraftedFeatureExtractor                             0.999344   
                   NoneFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                 0.999957   
                   amazon/chronos-t5-small                                 1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer             0.999764   
                   paris-noah/Mantis-8M                                    0.999974   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.033637   
LogisticRegression AutonLab/MOMENT-1-large                                    0.461957   
                   HandcraftedFeatureExtractor               

In [3]:
results_lopo = present_results(
    all_results,
    val_method="tacv",
    remove_xgboost=True,
    remove_chronos_small_from_test=True,
)

,Dataset,Side,Model,Resampling,Features,Aggregator,Validation,accuracy_score avg,accuracy_score sem,balanced_accuracy_score avg,balanced_accuracy_score sem,matthews_corrcoef avg,matthews_corrcoef sem,f1_score avg,f1_score sem,precision_score avg,precision_score sem,recall_score avg,recall_score sem,roc_auc_score avg,roc_auc_score sem
39,apsync,engagement_10sec_4thresh,DummyClassifier,None,HandcraftedFeatureExtractor,None,TACV,0.560551,0.050561,0.503783,0.005718,0.005368,0.011405,0.663263,0.110992,0.664136,0.190783,0.734895,0.219935,0.503433,0.007327
41,apsync,engagement_10sec_4thresh,LogisticRegression,None,AutonLab/MOMENT-1-large,MeanChanAggregator,TACV,0.607922,0.141020,0.567032,0.137903,0.000652,0.061786,0.737747,0.109768,0.660955,0.193180,0.894366,0.062690,0.498298,0.028745
36,apsync,engagement_10sec_4thresh,LogisticRegression,None,HandcraftedFeatureExtractor,None,TACV,0.667375,0.172993,0.594323,0.189552,0.015067,0.038193,0.759490,0.162466,0.660075,0.198181,0.919078,0.134397,0.498733,0.013390
35,apsync,engagement_10sec_4thresh,LogisticRegression,None,amazon/chronos-t5-large,MeanChanAggregator,TACV,0.649133,0.169867,0.593500,0.178338,0.001016,0.035938,0.759958,0.134301,0.663225,0.191739,0.929280,0.043862,0.503533,0.010989
40,apsync,engagement_10sec_4thresh,LogisticRegression,None,amazon/chronos-t5-small,MeanChanAggregator,TACV,0.626427,0.158589,0.591177,0.154383,0.039567,0.050727,0.743578,0.123521,0.667028,0.192590,0.892729,0.080608,0.513582,0.019573
38,apsync,engagement_10sec_4thresh,LogisticRegression,None,ibm-granite/granite-timeseries-patchtsmixer,MeanChanAggregator,TACV,0.661483,0.194475,0.600000,0.198000,0.000000,0.000000,0.779894,0.138221,0.661483,0.194475,1.000000,0.000000,0.500000,0.000000
37,apsync,engagement_10sec_4thresh,LogisticRegression,None,paris-noah/Mantis-8M,None,TACV,0.577785,0.059046,0.542561,0.071585,0.027412,0.112598,0.663833,0.092072,0.676391,0.180319,0.675645,0.078516,0.519523,0.071223


Friedman Test accuracy_score: statistic=5.978, pvalue=0.426
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.999999   
                   HandcraftedFeatureExtractor                                    0.675740   
                   amazon/chronos-t5-large                                        0.999812   
                   amazon/chronos-t5-small                                        0.963987   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.948498   
                   paris-noah/Mantis-8M                                           0.999991   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.999999   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.577610   
                   amazon/chronos-t5-large                                    0.998694   
                   amazon/chronos-t5-small                                    0.928962   
                   ibm-granite/granite-timeseries-patchtsmixer                0.905096   
                   paris-noah/Mantis-8M                                       1.000000   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.675740   
LogisticRegression AutonLab/MOMENT-1-large                                        0.577610   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        0.876775   
                   amazon/chronos-t5-small                                        0.994704   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.997234   
                   paris-noah/Mantis-8M                                           0.527684   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.999812   
LogisticRegression AutonLab/MOMENT-1-large                                    0.998694   
                   HandcraftedFeatureExtractor                                0.876775   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.997234   
                   ibm-granite/granite-timeseries-patchtsmixer                0.994704   
                   paris-noah/Mantis-8M                                       0.997234   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.963987   
LogisticRegression AutonLab/MOMENT-1-large                                    0.928962   
                   HandcraftedFeatureExtractor                                0.994704   
                   amazon/chronos-t5-large                                    0.997234   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                1.000000   
                   paris-noah/Mantis-8M

Friedman Test balanced_accuracy_score: statistic=2.196, pvalue=0.901
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.999991   
                   HandcraftedFeatureExtractor                                    0.963987   
                   amazon/chronos-t5-large                                        0.990645   
                   amazon/chronos-t5-small                                        0.905096   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.990645   
                   paris-noah/Mantis-8M                                           0.999949   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.999991   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.990645   
                   amazon/chronos-t5-large                                    0.998694   
                   amazon/chronos-t5-small                                    0.963987   
                   ibm-granite/granite-timeseries-patchtsmixer                0.998694   
                   paris-noah/Mantis-8M                                       1.000000   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.963987   
LogisticRegression AutonLab/MOMENT-1-large                                        0.990645   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        0.999991   
                   amazon/chronos-t5-small                                        0.999991   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.999991   
                   paris-noah/Mantis-8M                                           0.994704   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.990645   
LogisticRegression AutonLab/MOMENT-1-large                                    0.998694   
                   HandcraftedFeatureExtractor                                0.999991   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.999459   
                   ibm-granite/granite-timeseries-patchtsmixer                1.000000   
                   paris-noah/Mantis-8M                                       0.999459   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.905096   
LogisticRegression AutonLab/MOMENT-1-large                                    0.963987   
                   HandcraftedFeatureExtractor                                0.999991   
                   amazon/chronos-t5-large                                    0.999459   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.999459   
                   paris-noah/Mantis-8M

...

DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.928962   
                   HandcraftedFeatureExtractor                                    0.478211   
                   amazon/chronos-t5-large                                        0.013423   
                   amazon/chronos-t5-small                                        0.008059   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.339140   
                   paris-noah/Mantis-8M                                           0.223897   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.928962   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.984540   
                   amazon/chronos-t5-large                                    0.259158   
                   amazon/chronos-t5-small                                    0.191943   
                   ibm-granite/granite-timeseries-patchtsmixer                0.948498   
                   paris-noah/Mantis-8M                                       0.876775   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.478211   
LogisticRegression AutonLab/MOMENT-1-large                                        0.984540   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        0.766379   
                   amazon/chronos-t5-small                                        0.675740   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.999991   
                   paris-noah/Mantis-8M                                           0.999459   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.013423   
LogisticRegression AutonLab/MOMENT-1-large                                    0.259158   
                   HandcraftedFeatureExtractor                                0.766379   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.999999   
                   ibm-granite/granite-timeseries-patchtsmixer                0.876775   
                   paris-noah/Mantis-8M                                       0.948498   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.008059   
LogisticRegression AutonLab/MOMENT-1-large                                    0.191943   
                   HandcraftedFeatureExtractor                                0.675740   
                   amazon/chronos-t5-large                                    0.999999   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.807113   
                   paris-noah/Mantis-8M

Friedman Test recall_score: statistic=22.390, pvalue=0.001
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.999999   
                   HandcraftedFeatureExtractor                                    0.021791   
                   amazon/chronos-t5-large                                        0.675740   
                   amazon/chronos-t5-small                                        0.297637   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.096148   
                   paris-noah/Mantis-8M                                           0.096148   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.999999   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.013423   
                   amazon/chronos-t5-large                                    0.577610   
                   amazon/chronos-t5-small                                    0.223897   
                   ibm-granite/granite-timeseries-patchtsmixer                0.065133   
                   paris-noah/Mantis-8M                                       0.065133   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.021791   
LogisticRegression AutonLab/MOMENT-1-large                                        0.013423   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        0.675740   
                   amazon/chronos-t5-small                                        0.948498   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.998694   
                   paris-noah/Mantis-8M                                           0.998694   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.675740   
LogisticRegression AutonLab/MOMENT-1-large                                    0.577610   
                   HandcraftedFeatureExtractor                                0.675740   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    0.997234   
                   ibm-granite/granite-timeseries-patchtsmixer                0.928962   
                   paris-noah/Mantis-8M                                       0.928962   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.297637   
LogisticRegression AutonLab/MOMENT-1-large                                    0.223897   
                   HandcraftedFeatureExtractor                                0.948498   
                   amazon/chronos-t5-large                                    0.997234   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.998694   
                   paris-noah/Mantis-8M

Friedman Test roc_auc_score: statistic=19.527, pvalue=0.003
Nemenyi post-hoc test


DummyClassifier  \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    1.000000   
LogisticRegression AutonLab/MOMENT-1-large                                        0.999459   
                   HandcraftedFeatureExtractor                                    0.079418   
                   amazon/chronos-t5-large                                        0.065133   
                   amazon/chronos-t5-small                                        0.053044   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.163298   
                   paris-noah/Mantis-8M                                           0.065133   

                                                                    LogisticRegression  \
                                                               AutonLab/MOMENT-1-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.999459   
LogisticRegression AutonLab/MOMENT-1-large                                    1.000000   
                   HandcraftedFeatureExtractor                                0.223897   
                   amazon/chronos-t5-large                                    0.191943   
                   amazon/chronos-t5-small                                    0.163298   
                   ibm-granite/granite-timeseries-patchtsmixer                0.383365   
                   paris-noah/Mantis-8M                                       0.191943   

                                                                                            \
                                                               HandcraftedFeatureExtractor   
DummyClassifier    HandcraftedFeatureExtractor                                    0.079418   
LogisticRegression AutonLab/MOMENT-1-large                                        0.223897   
                   HandcraftedFeatureExtractor                                    1.000000   
                   amazon/chronos-t5-large                                        1.000000   
                   amazon/chronos-t5-small                                        0.999999   
                   ibm-granite/granite-timeseries-patchtsmixer                    0.999949   
                   paris-noah/Mantis-8M                                           1.000000   

                                                                                        \
                                                               amazon/chronos-t5-large   
DummyClassifier    HandcraftedFeatureExtractor                                0.065133   
LogisticRegression AutonLab/MOMENT-1-large                                    0.191943   
                   HandcraftedFeatureExtractor                                1.000000   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.999812   
                   paris-noah/Mantis-8M                                       1.000000   

                                                                                        \
                                                               amazon/chronos-t5-small   
DummyClassifier    HandcraftedFeatureExtractor                                0.053044   
LogisticRegression AutonLab/MOMENT-1-large                                    0.163298   
                   HandcraftedFeatureExtractor                                0.999999   
                   amazon/chronos-t5-large                                    1.000000   
                   amazon/chronos-t5-small                                    1.000000   
                   ibm-granite/granite-timeseries-patchtsmixer                0.999459   
                   paris-noah/Mantis-8M